### K Near Neighbors (KNN)

A técnica de KNN localiza os K primeiros vizinhos de um novo dado ponto. Coso os pontos próximos pertencem ao Grupo_i ele também pertencerá a este grupo. Pense que um rio separa duas regiões, e você abre um raio de 30 metros. Caso a margem direita ocupar mais este determinado círcuo, então o ponto onde você está pertense à margem direita.
<br><br>
Caso k for muito pequeno ocorre o overfiting, uma vez que acompanharemos sempre o vinho mais próximo, e como se acompanhássemos o ruído (linhas irregulares de uma fronteira). Se K aumentar, o novo ponto (vermelho) pegará a média da região, podendo mapear melhor a média da região. E se K for muito grande, estaremos sobre overfitting, e mapearemos muito mal o perfil da região ora em estudo.

Para se saber o ótimo k existe um cálculo de "Quantidade de Informação" (AIC - Akaike Information criterion, ou BIC - bayesian information criterion) que mostrará o melhor K. Os conceitos de AKI e BIC.

https://en.wikipedia.org/wiki/Akaike_information_criterion

![PCA](../figure/knn.png)

Nesse exemplo temos 5 registros da classe A e 5 registros da classe B, o objetivo é saber a qual classe o novo registro (bolinha vermelha) vai pertencer.

Dado um novo registro ele vai calcular a distância desse registro com todas as amostras da base de dados de treinamento.

Precisamos especificar um valor para o parâmetro K, ou seja, informar qual vai ser o número de vizinhos que serão comparados.

No exemplo temos K=3, então ele vai pegar somente as três amostras mais próximas para fazer a comparação.

Das 3 amostras de treinamento mais próximas do novo registro, 2 são da classe B e 1 da classe A, portanto como existem mais vizinhos da classe B, esse novo registro pertencerá a classe B.

texto de: https://minerandodados.com.br/machine-learning-na-pratica-knn-python/

In [ ]:
import os, sys
import pandas as pd # pandas e seu alias pd
import numpy as np  # numpy  e seu alias np

from   scipy import stats

from sklearn.decomposition import PCA

sys.path.insert(1, '../libs/')
from stat_lib import *

import seaborn as sns

import matplotlib.pyplot as plt # matplotlib e seu alias plt
%matplotlib inline

### Abrindo a tabela de Iris de Fisher & Anderson
  - vamos pegar as 4 colunas de dados 
  - vamos pegar 70% dos dados para treinamento
  - vamos pegar 30% dos dados para avaliação (testes)
  - por fim vamos calcular o AIC, para diferentes valores de K.

In [ ]:
fname = "../data/Iris.csv"
if os.path.exists(fname):
    df = pd.read_csv(fname)
else:
    print("Impossível abrir '%s'"%(fname))
    raise('Error')
    
print(df.shape)
df.columns = ['id', 'sepal_len', 'sepal_width', 'petal_len', 'petal_width', 'species']

df.head(3)

In [ ]:
data  = df[ ['sepal_len', 'sepal_width', 'petal_len', 'petal_width']]

colors = ['red', 'blue', 'green']
specColors = ['red']*50 + ['blue']*50 + ['green']*50

y = [0]*50 + [1]*50 + [2]*50
species = df.species.to_list()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.30)

In [ ]:
X_train.shape, X_test.shape

### Como temos 4 medidas (pétalas & sépalas x largura e comprimento)
### Normalizamos estes valores pois se um for de grande magnitude e dispersão, fará um viés com relação aos outros
### Normalizar (ou standardizar) significa subtrair a média e dividir pelo desvio padrão
  - Esta operação é similar ao que se faz na distribuiçao Z
  - Lembre-se que cada distribuição terá seu padrão, apesar das 4 se aproximarem da Normal
  - Neste caso folhas largas e compridas ficaram num canto do hiper-espaço
  - e folhas finas e curtas noutro canto do hiper-espaço

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=6)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

### Uma tabela de confusão é uma tabela que se pode analisar o que se acertou ou errou

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm
# print(classification_report(y_test, y_pred))

In [ ]:
def calculate_aic(n, mse, num_params):
    aic = n * log(mse) + 2 * num_params
    return aic

In [ ]:
N = np.sum(cm)
N

In [ ]:
cmp = cm / N
cmp

In [ ]:
def calc_ll(cm):
    cmp = cm / N

    ll = 0
    for i in range(3):
        for j in range(3):
            # print(i, j, cmp[i,j])
            if cm[i,j] > 0 and cmp[i,j] > 0:
                ll += cm[i,j] * np.log(cmp[i,j])
    return ll        

In [ ]:
calc_ll(cm)

In [ ]:
lls = []
KS = np.arange(4, 25)

ll_best = -99999
kbest = -1
cm_best = None

for K in KS:
    classifier = KNeighborsClassifier(n_neighbors=K)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    ll = calc_ll(cm)
    
    if ll > ll_best:
        ll_best = ll
        kbest = K
        cm_best = cm
    
    lls.append(ll)

In [ ]:
plt.plot(KS, lls)

plt.ylabel("log-likelihood (ll)")
plt.xlabel("K")
plt.title("KNN - K best choice");

In [ ]:
ll_best, kbest

In [ ]:
cm_best